In [1]:
from IPython.display import clear_output
%load_ext autoreload
%autoreload 2
import os
print(os.getcwd())
os.chdir('/home/vco/Projects/pm4py-dcr') # working directory should be pm4py-dcr (the one behind notebooks)
print(os.getcwd())

/home/vco/Projects/pm4py-dcr/notebooks
/home/vco/Projects/pm4py-dcr


In [7]:
import pm4py
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np

from copy import deepcopy
from scipy import stats
from pm4py.algo.discovery.dcr_discover import algorithm as alg
from pm4py.objects.dcr.exporter import exporter as dcr_exporter
import networkx as nx

In [3]:
A,B,X,Y,Z = 'A','B','X','Y','Z'
dcr = {'events': {A,B,X,Y,Z},
       'excludesTo': {
           X: {X,Y,A,B},
           Y: {Y,X,A},
           Z: {A},
           A: {X}
       }}

In [51]:
# Tijs example
dcr = {'events': {A,B,X,Y,Z},
       'excludesTo': {
           X: {Y,X,Z},
           Y: {X,Z,Y},
           Z: {Y,Z,X},
           A: {B,A,Z},
           B: {B,A}
       }}

In [52]:
cluster_pair = set()
for e1 in dcr['events']:
    if e1 in dcr['excludesTo']:
        for e2 in dcr['excludesTo'][e1]:
            if e2!=e1 and e2 in dcr['excludesTo'] and e1 in dcr['excludesTo'][e2]:
                cluster_pair.add((e1, e2))

In [53]:
cluster_pair

{('A', 'B'),
 ('B', 'A'),
 ('X', 'Y'),
 ('X', 'Z'),
 ('Y', 'X'),
 ('Y', 'Z'),
 ('Z', 'X'),
 ('Z', 'Y')}

In [55]:
# Strict based on both directions for both events
clusters = {}
# for i in range(len(dcr['events'])):
#     clusters[i] = set()
k=0
for (e1,e2) in cluster_pair:
    for (e3,e4) in cluster_pair:
        if e1!=e3 and e2!=e4:
            if e1==e4 and e2==e3:
                # they are in both directions (X,Y) <-> (Y,X)
                # now check if (e1,e2) has other pairs to cluster with, that respect:
                # (e1,e5) (e2,e5) (e1,e6) (e2,e5) OR (e5,e1) (e5,e2) (e6,e1) (e6,e2) are in the cluster
                if k != 0:
                    for (e5, e6) in cluster_pair:
                        if ((e1,e5) in cluster_pair and (e2,e5) in cluster_pair and (e1,e6) in cluster_pair and (e2,e5) in cluster_pair) or ((e5,e1) in cluster_pair and (e5,e2) in cluster_pair and (e6,e1) in cluster_pair and (e6,e2) in cluster_pair) and e1!=e5 and e2!=e6 and e1!=e6 and e2!=e5:
                            # now I want to create disjoint sets of events that are individual clusters
                            # here I get confused because I have to add new ones to the correct cluster
                            match_found = False
                            for j in range(k):
                                if e1 in clusters[j] and e2 in clusters[j]:
                                    clusters[j].add(e5)
                                    clusters[j].add(e6)
                                    match_found = True
                                    break
                            if match_found:
                                break
                    is_in_previous_cluster = 0
                    for j in range(k):
                        if e1 in clusters[j] and e2 in clusters[j]: #and set([e1,e2]).isdisjoint(clusters[j]):
                           is_in_previous_cluster = is_in_previous_cluster + 1
                    if is_in_previous_cluster == 0:
                        clusters[k] = set([e1,e2])
                        k = k + 1
                    else:
                        break
                elif k == 0:
                    clusters[k] = set([e1,e2])
                    k = k + 1
                    break

clusters

{0: {'X', 'Y', 'Z'}, 1: {'A', 'B'}}

In [56]:
# Relaxed based on connected components
G = nx.Graph()
G.add_edges_from(cluster_pair)
for connected_component in nx.connected_components(G):
    print(connected_component)

{'Z', 'Y', 'X'}
{'A', 'B'}
